## **LSTM implementation**

We implement the LSTM method in order to compare it to the ARIMA model.

### **Data Preparation**

We want to feed the data into an LSTM. So, we have to scale the time series and structure it into sequences.

* We begin by importing the necessary libraries for our study.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

* Then, we load the dataset and ensure your 'Close' column is available and transform the "date" column to datetime format.

In [ ]:
data = pd.read_csv("coin_Cardano.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')

* After that we select the Close column and scale it

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close_Scaled'] = scaler.fit_transform(data[['Close']])

* Next, we create sequences for LSTM

In [ ]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [ ]:
sequence_length = 30
sequences, targets = create_sequences(data['Close_Scaled'].values, sequence_length)

### **Split into Training and Testing Sets**



In [ ]:
train_size = int(len(sequences) * 0.8)
X_train, X_test = sequences[:train_size], sequences[train_size:]
y_train, y_test = targets[:train_size], targets[train_size:]

### **Build and Train the LSTM Model**

So we can predict the next close price.

* First, we define the LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(sequence_length, 1)))
model.add(Dense(1))  # Output layer
model.compile(optimizer='adam', loss='mean_squared_error')

* And then we train it

In [ ]:
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

history = model.fit(X_train_reshaped, y_train, epochs=20, batch_size=32, validation_data=(X_test_reshaped, y_test))

### **LSTM Evaluation**



* We begin by generating predictions

In [ ]:
y_pred = model.predict(X_test_reshaped)

* Then, we rescale the predictions back to their original scale

In [ ]:
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

* And then we calculate the metrics (MAE, MSE, MAPE)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
mape = np.mean(np.abs((y_test_rescaled - y_pred_rescaled) / y_test_rescaled)) * 100

print(f"LSTM Performance:\nMAE: {mae}\nMSE: {mse}\nMAPE: {mape}%")

### **Results Plots**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_test_rescaled, label='Actual Close Price')
plt.plot(y_pred_rescaled, label='LSTM Predicted Close Price')
plt.title('LSTM Forecast vs Actual')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()